 #### About
 1. CycleGAN attempts to learn a mapping from one dataset to another like a dataset of plants to dataset of faces.
 2. It does this with two generators G and F and two discriminators Dx and Dy.
 3. G attempts to turn dataset A into dataset B and F attempts to turn dataset B into dataset A.
 4. Dx is used to differentiate real images sampled from Dataset A and images generated by generator F. F is updated to get better at fooling Dx
 5. Dy is used to differentiate real images sampled from Dataset B and images generated by generator G. G is updated to get better at fooling Dy.
 6. Introduction of cycle loss is made to ensure an image that is yielded by generator can be mapped back to original image by other generator i.e F(G(x)) - x and G(F(y)) - y. Two cycles of Generators.
 7. Introduction of identity loss is made to help preserve tint, contrast and color shade. It ensures that when an image of target class is given, Generator yields the same image. F(x) - x and G(y) - y.
 8. A series of conv layers and residual layers are used in generator to map one image to another.
 9. PatchGAN architecture is used to classify images via Discriminator.
 10. Reflection padding is used in architecture instead of zero padding to reduce artifacts.
 11. The generator consists of encoder and decoder. It downsamples or encode the input image then interpret the encoding with series of residual blocks having skip connections. After this, Upsampling is being done to decode the representation to the size of the fake image.
 12. Instance normalisation is used instead of batch normalisation which results in better image generated by operating across image channels.
 13. Discriminator uses a PatchGAN in modification to the regular GAN architecture. The regular GAN maps an input image to single scalar output in range of 0 to 1 indicating the image being real or fake via probability. Whereas PatchGAN provides a matrix as output where each element of the matrix signifies whether its corresponding patch is real or fake. The receptive field of discriminator is 70*70 in Cycle GAN.
 ![cycle_gan](cycle_gan.png)
 14. To reduce model oscillation, Replay buffer method is implemented. The discriminator is updated using a history of generated images rather than the ones produced by the latest generators. An image buffer to store previous created images is stored.

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [14]:
#importing modules
import os
import sys
import numpy as np
import time
import glob
import random
from PIL import Image
import math
import itertools
import scipy
import copy
import shutil

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image, make_grid
from torchvision import transforms
from torchvision import datasets
import torch.optim as optim
from torchsummary import summary

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython.display import clear_output
%matplotlib inline



In [ ]:
#unzipping the dataset
os.chdir('/content/drive/MyDrive/datasets/')
!unzip apple2orange.zip

In [20]:
#utility functions
def create_dir(path):
    if not isinstance(path, (list,tuple)):
        paths = [path]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path)

def convert_to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image

dataset_dir = '/home/suraj/ClickUp/Jan-Feb/data/apple2orange/'
apple_train_dir = dataset_dir + '/trainA/'
orange_train_dir = dataset_dir + '/trainB/'
apple_val_dir = dataset_dir + '/testA/'
orange_val_dir = dataset_dir + '/testB/'


In [21]:
class GanDataset(Dataset):
    def __init__(self, trainA_path,trainB_path, transform = None, unaligned = False):
        self.transform = transform
        self.unaligned = unaligned

        self.file_names_A = sorted(glob.glob(trainA_path+"*.*"))
        self.file_names_B = sorted(glob.glob(trainB_path+"*.*"))

    
    def __getitem__(self,index):
        image_A = Image.open(self.file_names_A[index])
        if self.unaligned:
            image_B = Image.open(self.file_names_B[random.randint(0,len(self.file_names_B)-1)])
        else:
            image_B = Image.open(self.file_names_B[index])
        # if image is grayscale , then convert to RGB
        if image_A.mode!="RGB":
            image_A = convert_to_rgb(image_A)
        if image_B.mode!="RGB":
            image_B = convert_to_rgb(image_B)

        if self.transform:
            image_A = self.transform(image_A)
            image_B = self.transform(image_B)

        return {"first_image":image_A, "second_image":image_B}

    
    def __len__(self):
        return max(len(self.file_names_A),len(self.file_names_B))

In [22]:


transform = transforms.Compose([
    transforms.Resize((128, 128), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


train_dataset = GanDataset(apple_train_dir,orange_train_dir,transform= transform)
val_dataset = GanDataset(apple_val_dir,orange_val_dir,transform= transform)

/home/suraj/anaconda3/envs/torch_dl/lib/python3.8/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [23]:
train_dataset.__getitem__(6)

{'first_image': tensor([[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -0.9059, -0.7725, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -0.9922]],
 
         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -0.9059, -0.7725, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -0.9922]],
 
         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1

In [24]:
val_dataset.__getitem__(5)

{'first_image': tensor([[[0.5059, 0.5059, 0.5059,  ..., 0.4980, 0.4980, 0.4980],
          [0.5059, 0.5059, 0.5059,  ..., 0.4980, 0.4980, 0.4980],
          [0.5059, 0.5059, 0.5059,  ..., 0.4980, 0.4980, 0.4980],
          ...,
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059]],
 
         [[0.5059, 0.5059, 0.5059,  ..., 0.5137, 0.5137, 0.5137],
          [0.5059, 0.5059, 0.5059,  ..., 0.5137, 0.5137, 0.5137],
          [0.5059, 0.5059, 0.5059,  ..., 0.5137, 0.5137, 0.5137],
          ...,
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059]],
 
         [[0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059, 0.5059,  ..., 0.5059, 0.5059, 0.5059],
          [0.5059, 0.5059

In [27]:
# creating data laoder
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True,num_workers=4)

In [28]:
for batch in train_loader:
    print(batch['first_image'].shape, batch['second_image'].shape)
    break

torch.Size([32, 3, 128, 128]) torch.Size([32, 3, 128, 128])


In [ ]:
#visualizing